In [7]:
"""Demonstrates the ee.Classifier.amnhMaxent method."""

import ee
import geemap

# Authenticates to the Earth Engine servers.
# ee.Authenticate()
# Initializes the client library.
# ee.Initialize()



In [2]:
# Create some sample species presence/absence training data.
training_data = ee.FeatureCollection([
    # Species present points.
    ee.Feature(ee.Geometry.Point([-122.39567, 38.02740]), {'presence': 1}),
    ee.Feature(ee.Geometry.Point([-122.68560, 37.83690]), {'presence': 1}),
    # Species absent points.
    ee.Feature(ee.Geometry.Point([-122.59755, 37.92402]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.47137, 37.99291]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.52905, 37.85642]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.03010, 37.66660]), {'presence': 0})
])


In [3]:
# Import a Landsat 8 image and select the reflectance bands.
image = (ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_044034_20200606')
         .select(['SR_B[1-7]'])
         .multiply(0.0000275).add(-0.2))  # Apply scaling factors.

# Sample the image at the location of the points.
training = image.sampleRegions(**{
    'collection': training_data,
    'scale': 30
})




In [4]:
# Define and train a Maxent classifier from the image-sampled points.
classifier = ee.Classifier.amnhMaxent().train(**{
    'features': training,
    'classProperty': 'presence',
    'inputProperties': image.bandNames()
})



In [5]:
# Classify the image using the Maxent classifier.
image_classified = image.classify(classifier)


In [8]:
Map = geemap.Map()


In [13]:
Map.addLayer(image_classified)
Map.addLayer(training_data)
Map


Map(bottom=25685.0, center=[37.54022177661216, -121.81362845010851], controls=(WidgetControl(options=['positio…